### So the function to match arguments is needed
#### It has two use cases:
- mock.Setup
- mock.Verify

#### Moq Sample

```CSharp
var mock = new Mock<IFoo>();
mock.Object.Echo(1);
mock.Verify(m => m.Echo(It.Is<int>(i => i != 2)));

In [1]:
// Let's guess signatures
public class Arg
{
  public static T IsAny<T>() => throw new InvalidOperationException();
  public static T Is<T>(Func<T, bool> matcher) => throw new InvalidOperationException();
}

The below script needs to be able to find the current output cell; this is an easy method to get it.

### How Moq Did it?
#### Demo
Debug Verify_if_successful_marks_only_matched_invocations_as_verified
With breakpoint in MatcherFactory.CreateMatcher

### All the Code From Previous Workbook

In [2]:
#r "nuget: Castle.Core"

using System.Linq.Expressions;
using Castle.DynamicProxy;


public interface IMockInvocation
{
  bool IsMatch(IMockInvocation invocation);
}

public class MockInvocationSetup : IMockInvocation
{
  private readonly Expression _expression;
  public MockInvocationSetup(Expression expression)
  {
    _expression = expression;
  }
  public bool IsMatch(IMockInvocation invocation) => true;
}

public class MockInvocation : IMockInvocation
{
  private readonly IInvocation _invocation;
  public MockInvocation(IInvocation invocation)
  {
    _invocation = invocation;
  }
  
  public bool IsMatch(IMockInvocation invocation) => true;
}

public class MockObject<T>  where T : class 
{
  private readonly Lazy<T> _lazy;
  private readonly MockInterceptor _interceptor = new();
  
  public MockObject() 
  {
    _lazy = new Lazy<T>(CreateInstance);
  }

  public void Verify(Expression<Action<T>> action, Executed times)
  {
    var invocationMatch = new MockInvocationSetup(action);
    times.Verify(_interceptor.Invocations.Count(x => x.IsMatch(invocationMatch)));
  }

  public T Object => _lazy.Value;

  private T CreateInstance() => FactoryProvider.Factory.GetProxy<T>(_interceptor);
}

public class Executed
{
  private readonly Func<int, bool> _validate;
  private readonly string _message;

  public Executed(Func<int, bool> validate, string message)
  {
    _validate = validate;
    _message = message;
  }

  public void Verify(int count)
  {
    if (!_validate.Invoke(count))
    {
      throw new Exception($"Executed wrong number of times. Expected: {_message} Executed: {count}");
    }
  }

  public static Executed Once => new (x => x == 1, "Once");
  public static Executed Never => new (x => x == 0, "Never");
  public static Executed AtLeast(int times) => new (x => x >= times, $"At least {times}");
  public static Executed AtMost(int times) => new (x => x <= times, $"At most {times}");
}



public class MockInterceptor : StandardInterceptor
{
  private readonly List<IMockInvocation> _invocations = new();

  public IReadOnlyList<IMockInvocation> Invocations => _invocations;

  protected override void PreProceed(IInvocation invocation)
  {
    _invocations.Add(new MockInvocation(invocation));
    base.PreProceed(invocation);
  }

  protected override void PerformProceed(IInvocation invocation)
  {
  }
}


public class ProxyFactory
{
  private readonly ProxyGenerator _generator = new ();

  public T GetProxy<T>(IInterceptor interceptor) where T:class => _generator.CreateInterfaceProxyWithoutTarget<T>(interceptor);
}

public static class FactoryProvider 
{
  public static ProxyFactory Factory {get;} = new ProxyFactory();
}

Installed Packages Castle.Core, 5.1.1

### So

In [5]:
public interface IFoo
{
  void M(int n);
  void A();
}

var myMock = new MockObject<IFoo>();
myMock.Object.A();
myMock.Verify(obj => obj.M(Arg.Is<int>(x => x == 10)), Executed.Once);

In [6]:
Expression<Action<IFoo>> f = obj => obj.M(Arg.Is<int>(x => x == 10));
f

obj => obj.M(Is(x => (x == 10))) Type System.Action<Submission#7+IFoo> NodeType Lambda Parameters index value 0 obj Type Submission#7+IFoo NodeType Parameter Name obj IsByRef False CanReduce False Name <null> Body obj.M(Is(x => (x == 10))) ArgumentCount 1 NodeType Call Type System.Void Method Void M(Int32) Name M DeclaringType Submission#7+IFoo ReflectedType Submission#7+IFoo MemberType Method MetadataToken 100663302 Module ℛ*a61a45fc-86bc-4bf9-89e2-736be1e276ee#1-7.dll MDStreamVersion 131072 FullyQualifiedName <Unknown> ModuleVersionId 412e7c99-c719-40e1-8519-2b27abc828f1 MetadataToken 1 ScopeName ℛ*a61a45fc-86bc-4bf9-89e2-736be1e276ee#1-7.dll Name <Unknown> Assembly ℛ*a61a45fc-86bc-4bf9-89e2-736be1e276ee#1-7, Version=0.0.0.0, Culture=neutral, PublicKeyToken=null CodeBase file:///C:/Program Files/dotnet/shared/Microsoft.NETCore.App/7.0.4/System.Private.CoreLib.dll FullName ℛ*a61a45fc-86bc-4bf9-89e2-736be1e276ee#1-7, Version=0.0.0.0, Culture=neutral, PublicKeyToken=null EntryPoint <null> DefinedTypes index value 0 Microsoft.CodeAnalysis.EmbeddedAttribute 1 System.Runtime.CompilerServices.RefSafetyRulesAttribute 2 Submission#7 3 Submission#7+IFoo 4 Submission#7+< >d__0 5 Submission#7+<>c IsCollectible False ManifestModule ℛ*a61a45fc-86bc-4bf9-89e2-736be1e276ee#1-7.dll MDStreamVersion 131072 FullyQualifiedName <Unknown> ModuleVersionId 412e7c99-c719-40e1-8519-2b27abc828f1 MetadataToken 1 ScopeName ℛ*a61a45fc-86bc-4bf9-89e2-736be1e276ee#1-7.dll Name <Unknown> Assembly ℛ*a61a45fc-86bc-4bf9-89e2-736be1e276ee#1-7, Version=0.0.0.0, Culture=neutral, PublicKeyToken=null ModuleHandle System.ModuleHandle CustomAttributes [ [System.Security.UnverifiableCodeAttribute()], [System.Runtime.CompilerServices.RefSafetyRulesAttribute((Int32)11)] ] ReflectionOnly False Location ImageRuntimeVersion v4.0.30319 GlobalAssemblyCache False HostContext 0 IsDynamic False ExportedTypes index value 0 Microsoft.CodeAnalysis.EmbeddedAttribute 1 System.Runtime.CompilerServices.RefSafetyRulesAttribute 2 Submission#7 3 Submission#7+IFoo 4 Submission#7+< >d__0 5 Submission#7+<>c IsFullyTrusted True CustomAttributes index value 0 [System.Runtime.CompilerServices.CompilationRelaxationsAttribute((Int32)8)] 1 [System.Runtime.CompilerServices.RuntimeCompatibilityAttribute(WrapNonExceptionThrows = True)] 2 [System.Diagnostics.DebuggableAttribute((System.Diagnostics.DebuggableAttribute+DebuggingModes)263)] EscapedCodeBase file:///C:/Program%20Files/dotnet/shared/Microsoft.NETCore.App/7.0.4/System.Private.CoreLib.dll Modules index value 0 ℛ*a61a45fc-86bc-4bf9-89e2-736be1e276ee#1-7.dll SecurityRuleSet None ModuleHandle System.ModuleHandle MDStreamVersion 131072 CustomAttributes index value 0 [System.Security.UnverifiableCodeAttribute()] Constructor Void .ctor() ConstructorArguments [ ] NamedArguments [ ] AttributeType System.Security.UnverifiableCodeAttribute 1 [System.Runtime.CompilerServices.RefSafetyRulesAttribute((Int32)11)] Constructor Void .ctor(Int32) ConstructorArguments [ (Int32)11 ] NamedArguments [ ] AttributeType System.Runtime.CompilerServices.RefSafetyRulesAttribute IsSecurityCritical True IsSecuritySafeCritical False IsSecurityTransparent False MethodHandle System.RuntimeMethodHandle Value 140729149643912 Attributes PrivateScope, Public, Virtual, HideBySig, VtableLayoutMask, Abstract CallingConvention Standard, HasThis ReturnType System.Void ReturnTypeCustomAttributes Void ParameterType System.Void Name <null> HasDefaultValue True DefaultValue <null> RawDefaultValue <null> MetadataToken 134217728 Attributes None Member Void M(Int32) Name M DeclaringType Submission#7+IFoo ReflectedType Submission#7+IFoo MemberType Method MetadataToken 100663302 Module ℛ*a61a45fc-86bc-4bf9-89e2-736be1e276ee#1-7.dll MDStreamVersion 131072 FullyQualifiedName <Unknown> ModuleVersionId 412e7c99-c719-40e1-8519-2b27abc828f1 MetadataToken 1 ScopeName ℛ*a61a45fc-86bc-4bf9-89e2-736be1e276ee#1-7.dll Name <Unknown> Assembly ℛ*a61a45fc-86bc-4bf9-89e2-736be1e276ee#1-7, Version=0.0.0.0, Cu

In [7]:
public class Visitor : ExpressionVisitor
{
  protected override Expression VisitMethodCall(MethodCallExpression node)
  {
    Console.WriteLine(node.Method.Name);
    return base.VisitMethodCall(node);
  }
}

In [8]:
new Visitor().Visit(f);

M
Is


In [10]:
using System.Reflection;

public class Visitor : ExpressionVisitor
{
  public MethodInfo Method { get; private set; }
  protected override Expression VisitMethodCall(MethodCallExpression node)
  {
    var result = base.VisitMethodCall(node);
    
    Method = node.Method;
    return result;
  }
}


In [13]:
var visitor = new Visitor();
visitor.Visit(f);
//visitor.Method
((MethodCallExpression)((MethodCallExpression)f.Body).Arguments[0]).Arguments[0]

x => (x == 10) Type System.Func<System.Int32,System.Boolean> NodeType Lambda Parameters index value 0 x Type System.Int32 NodeType Parameter Name x IsByRef False CanReduce False Name <null> Body (x == 10) Type System.Boolean NodeType Equal CanReduce False Right 10 Type System.Int32 NodeType Constant Value 10 CanReduce False Left x Type System.Int32 NodeType Parameter Name x IsByRef False CanReduce False Method <null> Conversion <null> IsLifted False IsLiftedToNull False ReturnType System.Boolean TailCall False CanReduce False

In [14]:
typeof(Arg).GetMethods(BindingFlags.Static | BindingFlags.Public).Single(x => x.Name == "Is")

T Is[T](System.Func`2[T,System.Boolean]) Name Is DeclaringType Submission#3+Arg ReflectedType Submission#3+Arg MemberType Method MetadataToken 100663303 Module ℛ*a61a45fc-86bc-4bf9-89e2-736be1e276ee#1-3.dll MDStreamVersion 131072 FullyQualifiedName <Unknown> ModuleVersionId a1c5eea7-6121-4061-bba1-e3bb8a99bef0 MetadataToken 1 ScopeName ℛ*a61a45fc-86bc-4bf9-89e2-736be1e276ee#1-3.dll Name <Unknown> Assembly ℛ*a61a45fc-86bc-4bf9-89e2-736be1e276ee#1-3, Version=0.0.0.0, Culture=neutral, PublicKeyToken=null CodeBase file:///C:/Program Files/dotnet/shared/Microsoft.NETCore.App/7.0.4/System.Private.CoreLib.dll FullName ℛ*a61a45fc-86bc-4bf9-89e2-736be1e276ee#1-3, Version=0.0.0.0, Culture=neutral, PublicKeyToken=null EntryPoint <null> DefinedTypes index value 0 Microsoft.CodeAnalysis.EmbeddedAttribute 1 System.Runtime.CompilerServices.RefSafetyRulesAttribute 2 Submission#3 3 Submission#3+Arg 4 Submission#3+<<Initialize>>d__0 IsCollectible False ManifestModule ℛ*a61a45fc-86bc-4bf9-89e2-736be1e276ee#1-3.dll MDStreamVersion 131072 FullyQualifiedName <Unknown> ModuleVersionId a1c5eea7-6121-4061-bba1-e3bb8a99bef0 MetadataToken 1 ScopeName ℛ*a61a45fc-86bc-4bf9-89e2-736be1e276ee#1-3.dll Name <Unknown> Assembly ℛ*a61a45fc-86bc-4bf9-89e2-736be1e276ee#1-3, Version=0.0.0.0, Culture=neutral, PublicKeyToken=null CodeBase file:///C:/Program Files/dotnet/shared/Microsoft.NETCore.App/7.0.4/System.Private.CoreLib.dll FullName ℛ*a61a45fc-86bc-4bf9-89e2-736be1e276ee#1-3, Version=0.0.0.0, Culture=neutral, PublicKeyToken=null EntryPoint <null> DefinedTypes index value 0 Microsoft.CodeAnalysis.EmbeddedAttribute 1 System.Runtime.CompilerServices.RefSafetyRulesAttribute 2 Submission#3 3 Submission#3+Arg 4 Submission#3+< >d__0 IsCollectible False ManifestModule ℛ*a61a45fc-86bc-4bf9-89e2-736be1e276ee#1-3.dll MDStreamVersion 131072 FullyQualifiedName <Unknown> ModuleVersionId a1c5eea7-6121-4061-bba1-e3bb8a99bef0 MetadataToken 1 ScopeName ℛ*a61a45fc-86bc-4bf9-89e2-736be1e276ee#1-3.dll Name <Unknown> Assembly ℛ*a61a45fc-86bc-4bf9-89e2-736be1e276ee#1-3, Version=0.0.0.0, Culture=neutral, PublicKeyToken=null ModuleHandle System.ModuleHandle CustomAttributes [ [System.Security.UnverifiableCodeAttribute()], [System.Runtime.CompilerServices.RefSafetyRulesAttribute((Int32)11)] ] ReflectionOnly False Location ImageRuntimeVersion v4.0.30319 GlobalAssemblyCache False HostContext 0 IsDynamic False ExportedTypes index value 0 Microsoft.CodeAnalysis.EmbeddedAttribute 1 System.Runtime.CompilerServices.RefSafetyRulesAttribute 2 Submission#3 3 Submission#3+Arg 4 Submission#3+< >d__0 IsFullyTrusted True CustomAttributes index value 0 [System.Runtime.CompilerServices.CompilationRelaxationsAttribute((Int32)8)] 1 [System.Runtime.CompilerServices.RuntimeCompatibilityAttribute(WrapNonExceptionThrows = True)] 2 [System.Diagnostics.DebuggableAttribute((System.Diagnostics.DebuggableAttribute+DebuggingModes)263)] EscapedCodeBase file:///C:/Program%20Files/dotnet/shared/Microsoft.NETCore.App/7.0.4/System.Private.CoreLib.dll Modules index value 0 ℛ*a61a45fc-86bc-4bf9-89e2-736be1e276ee#1-3.dll SecurityRuleSet None ModuleHandle System.ModuleHandle MDStreamVersion 131072 CustomAttributes index value 0 [System.Security.UnverifiableCodeAttribute()] Constructor Void .ctor() ConstructorArguments [ ] NamedArguments [ ] AttributeType System.Security.UnverifiableCodeAttribute 1 [System.Runtime.CompilerServices.RefSafetyRulesAttribute((Int32)11)] Constructor Void .ctor(Int32) ConstructorArguments [ (Int32)11 ] NamedArguments [ ] AttributeType System.Runtime.CompilerServices.RefSafetyRulesAttribute ReflectionOnly False Location ImageRuntimeVersion v4.0.30319 GlobalAssemblyCache False HostContext 0 IsDynamic False ExportedTypes index value 0 Microsoft.CodeAnalysis.EmbeddedAttribute 1 System.Runtime.CompilerServices.RefSafetyRulesAttribute 2 Submission#3 3 Submission#3+Arg 4 Submission#3+<<Initialize>>d__0 IsFullyTrusted True CustomAttributes index value 0 [System.Runtime.CompilerServices.CompilationRelaxa